- Better performance of the result
- The model should not be bigger in size

In [59]:
import matplotlib.pyplot as plt
import pandas as pd

import PIL.Image as Image
from IPython.display import Image, display
import matplotlib.cm as cm
import tensorflow as tf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
import numpy as np

In [60]:
directory = "data/"

BATCH_SIZE = 32
IMG_SIZE = (192, 192)

validation_dataset = tf.keras.utils.image_dataset_from_directory(directory,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            shuffle=True,
                                                            seed=999,
                                                            validation_split=0.20,
                                                            subset = "validation")

val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches)

Found 826 files belonging to 2 classes.
Using 165 files for validation.


In [61]:
loaded_model_1 = tf.keras.models.load_model('models/model1.keras')

loss, accuracy = loaded_model_1.evaluate(test_dataset)
print('Test accuracy :', accuracy)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.8991 - loss: 0.4369
Test accuracy : 0.8727272748947144


In [62]:
loaded_model_2 = tf.keras.models.load_model('models/model2.keras')

loss, accuracy = loaded_model_2.evaluate(test_dataset)
print('Test accuracy :', accuracy)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.7677 - loss: 0.5258
Test accuracy : 0.7878788113594055


In [63]:
def average_models(models):
    new_model = tf.keras.models.clone_model(models[0])
    new_weights = [np.mean([model.get_weights()[i] for model in models], axis=0) for i in range(len(models[0].get_weights()))]
    new_model.set_weights(new_weights)
    return new_model

models = [loaded_model_1, loaded_model_2]
average_model = average_models(models)

base_learning_rate = 0.0001

average_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

average_model.summary()

loss, average_model_accuracy = average_model.evaluate(test_dataset)
print('Test accuracy :', average_model_accuracy)


Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (None, 192, 192, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_192            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.8903 - loss: 0.4819
Test accuracy : 0.8909090757369995
